In [32]:
import os
import base64
import json
from io import BytesIO
from pdf2image import convert_from_path
from openai import AsyncOpenAI
from tqdm.asyncio import tqdm_asyncio
import pandas as pd
from config import settings

In [21]:
openai_client = AsyncOpenAI(api_key=settings.OPENAI_API_KEY)

In [22]:
def pdf_to_base64_images(pdf_path):
    imgs = convert_from_path(pdf_path, fmt='png')
    base64_imgs = []
    for image in imgs:
        buffered = BytesIO()
        image.save(buffered, format='PNG')
        img_str = base64.b64encode(buffered.getvalue()).decode()
        base64_imgs.append(img_str)
    
    return base64_imgs

In [23]:
SYSTEM_PROMPT = """You are an AI assistant specialized in extracting information from invoice images for the company "TechNova Solutions, Inc.". Your task is to analyze the given invoice image and extract the relevant information into a structured JSON format. If a field is not present in the invoice or cannot be determined leave it as "null".

Extract the information into the following JSON format:
```json
{
  "invoice_number": "string",
  "invoice_date": "YYYY-MM-DD",
  "invoice_type": "incoming | outgoing",
  "issuer": {
    "name": "string",
    "address": "string",
    "phone": "string",
    "email": "string"
  },
  "recipient": {
    "name": "string",
    "address": "string",
    "phone": "string",
    "email": "string"
  },
  "invoice_items": [
    {
      "description": "string",
      "total": "number"
    }
  ],
  "subtotal": "number",
  "tax_rate": "number (percentage)",
  "tax": "number",
  "total": "number",
  "terms": "string"
}
```

For the property "invoice_type" set the value "incoming" or "outgoing" from the point of view of the company "TechNova Solutions, Inc.". If this company does not appear as issuer or recipient, set it to "null".
"""

In [24]:
async def extract_invoice_data(base64_img):
    response = await openai_client.chat.completions.create(
        model='gpt-4o',
        response_format={ 'type': 'json_object' },
        messages=[
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': [
                {'type': 'image_url', 'image_url': {'url': f'data:image/png;base64,{base64_img}'}}
            ]}
        ],
        temperature=0.1,
    )
    return response.choices[0].message.content

In [39]:
INVOICES_DIR = 'data/invoices'

tasks = []
print('Processing the invoices:')
for filename in sorted(os.listdir(INVOICES_DIR)):
    if filename.endswith('.pdf'):
        print(filename)
        pdf_path = os.path.join(INVOICES_DIR, filename)
        base64_images = pdf_to_base64_images(pdf_path)
        task = extract_invoice_data(base64_images[0])
        tasks.append(task)
invoices_json = await tqdm_asyncio.gather(*tasks)
invoices_data = [json.loads(invoices_json) for invoices_json in invoices_json]

Processing the invoices:
2024-001.pdf
2024-002.pdf
2024-003.pdf
CS-9876.pdf
CS-9897.pdf
DT-45678.pdf
OS-112233.pdf


100%|██████████| 7/7 [00:15<00:00,  2.24s/it]


In [42]:
with open('data/invoices.json', 'w') as f:
    json.dump(invoices_data, f)

In [43]:
with open('data/invoices.json', 'r') as f:
    invoices_data = json.load(f)

In [177]:
def flatten_invoice(invoice):
    flat = {
        'Invoice Number': invoice['invoice_number'],
        'Invoice Date': invoice['invoice_date'],
        'Invoice Type': invoice['invoice_type'],
        'Issuer Name': invoice['issuer']['name'],
        'Issuer Address': invoice['issuer']['address'],
        'Issuer Phone': invoice['issuer']['phone'],
        'Issuer Email': invoice['issuer']['email'],
        'Recipient Name': invoice['recipient']['name'],
        'Recipient Address': invoice['recipient']['address'],
        'Recipient Phone': invoice['recipient']['phone'],
        'Recipient Email': invoice['recipient']['email'],
        'Subtotal': invoice['subtotal'],
        'Tax rate': invoice['tax_rate'],
        'Tax': invoice['tax'],
        'Total': invoice['total'],
        'Terms': invoice['terms']
    }
    
    for i, item in enumerate(invoice['invoice_items'], 1):
        flat[f'Item {i} Description'] = item['description']
        flat[f'Item {i} Total'] = item['total']
    
    return flat

In [178]:
def create_invoices_df(invoices_data):
    flattened_invoices = [flatten_invoice(invoice) for invoice in invoices_data]
    invoices_df = pd.DataFrame(flattened_invoices)
    invoices_df['Invoice Date'] = pd.to_datetime(invoices_df['Invoice Date'])
    invoices_df.insert(2, 'Year-Month', invoices_df['Invoice Date'].dt.to_period('M'))
    return invoices_df

In [194]:
def perform_financial_analysis(invoices_df):
    summary_ds = pd.Series(dtype='object')

    # Summary Analysis
    summary_ds['Period'] = f"{invoices_df['Invoice Date'].min().strftime('%Y-%m')} to {invoices_df['Invoice Date'].max().strftime('%Y-%m')}"
    summary_ds['Total Invoices'] = len(invoices_df)

    revenue = invoices_df[invoices_df['Invoice Type'] == 'outgoing']['Total'].sum()
    costs = invoices_df[invoices_df['Invoice Type'] == 'incoming']['Total'].sum()
    summary_ds['Total Revenue'] = revenue
    summary_ds['Total Costs'] = costs
    summary_ds['Profit'] = revenue - costs
    summary_ds['Profit Margin'] = ((revenue - costs) / revenue if revenue > 0 else 0).round(2)

    total_tax_collected = invoices_df[invoices_df['Invoice Type'] == 'outgoing']['Tax'].sum()
    total_tax_paid = invoices_df[invoices_df['Invoice Type'] == 'incoming']['Tax'].sum()
    summary_ds['Tax Collected'] = total_tax_collected
    summary_ds['Tax Paid'] = total_tax_paid
    summary_ds['Net Tax'] = total_tax_collected - total_tax_paid

    top_customer = invoices_df[invoices_df['Invoice Type'] == 'outgoing'].groupby('Recipient Name')['Total'].sum().idxmax()
    top_provider = invoices_df[invoices_df['Invoice Type'] == 'incoming'].groupby('Issuer Name')['Total'].sum().idxmax()
    summary_ds['Top Customer'] = top_customer
    summary_ds['Top Provider'] = top_provider

    # Monthly Analysis
    monthly_df = invoices_df.groupby('Year-Month').agg(**{
        'Invoices': ('Invoice Number', 'count'),
        'Revenue': ('Total', lambda x: x[invoices_df['Invoice Type'] == 'outgoing'].sum()),
        'Costs': ('Total', lambda x: x[invoices_df['Invoice Type'] == 'incoming'].sum()),
        'Tax Collected': ('Tax', lambda x: x[invoices_df['Invoice Type'] == 'outgoing'].sum()),
        'Tax Paid': ('Tax', lambda x: x[invoices_df['Invoice Type'] == 'incoming'].sum())
    })
    monthly_df = monthly_df.reset_index()
    monthly_df['Year-Month'] = monthly_df['Year-Month'].astype(str)
    monthly_df.insert(4, 'Profit', monthly_df['Revenue'] - monthly_df['Costs'])
    monthly_df.insert(5, 'Profit Margin', (monthly_df['Profit'] / monthly_df['Revenue']).round(2))
    monthly_df['Net Tax'] = monthly_df['Tax Collected'] - monthly_df['Tax Paid']

    return summary_ds, monthly_df.T

In [201]:
def create_excel(invoices_df, summary_ds, monthly_df, filepath='data/invoices.xlsx'):
    with pd.ExcelWriter(filepath, engine='xlsxwriter', datetime_format='YYYY-MM-DD') as writer:
        # Write invoice data sheet
        invoices_df.to_excel(writer, sheet_name='Invoices', index=False)

        # Create analysis sheet
        workbook = writer.book
        worksheet = workbook.add_worksheet('Financial Analysis')

        # Define formats
        title_format = workbook.add_format({'bold': True, 'font_size': 14})
        money_format = workbook.add_format({'num_format': '#,##0.00'})
        percent_format = workbook.add_format({'num_format': '0%'})

        # Write summary analysis
        worksheet.write('A1', 'SUMMARY', title_format)
        summary_ds.to_frame().to_excel(writer, sheet_name='Financial Analysis', startrow=1, startcol=0, header=False)

        # Write monthly analysis
        worksheet.write('A16', 'MONTHLY', title_format)
        monthly_df.to_excel(writer, sheet_name='Financial Analysis', startrow=16, startcol=0, header=False)

        # Formatting
        for row in [4, 5, 6, 8, 9, 10, 18, 19, 20, 22, 23, 24]:
            worksheet.set_row(row, None, money_format)
        worksheet.set_row(7, None, percent_format)
        worksheet.set_row(21, None, percent_format)

        # Create chart
        chart = workbook.add_chart({'type': 'line'})

        num_cols = monthly_df.shape[1]
        for i, metric in enumerate(['Revenue', 'Costs', 'Profit']):
            chart.add_series({
                'name': metric,
                'categories': ['Financial Analysis', 16, 1, 16, num_cols],
                'values': ['Financial Analysis', 18 + i, 1, 18 + i, num_cols],
            })

        chart.set_x_axis({'name': 'Month'})
        chart.set_y_axis({'name': 'Amount', 'major_gridlines': {'visible': False}})

        worksheet.insert_chart('F2', chart)

In [195]:
with open('data/invoices.json', 'r') as f:
    invoices_data = json.load(f)

invoices_df = create_invoices_df(invoices_data)
summary_ds, monthly_df = perform_financial_analysis(invoices_df)
create_excel(invoices_df, summary_ds, monthly_df)

Period            2024-01 to 2024-02
Total Invoices                     7
Total Revenue               194757.5
Total Costs                 20980.51
Profit                     173776.99
Profit Margin                   0.89
Tax Collected                15257.5
Tax Paid                     1730.51
Net Tax                     13526.99
Top Customer         GreenGrow Farms
Top Provider            DevTools Pro
dtype: object

,0,1
Year-Month,2024-01,2024-02
Invoices,4,3
Revenue,108500.0,86257.5
Costs,13846.88,7133.63
Profit,94653.12,79123.87
Profit Margin,0.87,0.92
Tax Collected,8500.0,6757.5
Tax Paid,1096.88,633.63
Net Tax,7403.12,6123.87
